In [9]:
# ddg imports
from ddgclib import *
from ddgclib._complex import *
from ddgclib._curvatures import * #plot_surface#, curvature
from ddgclib._capillary_rise_flow import * #plot_surface#, curvature
from ddgclib._eos import *
from ddgclib._misc import *
from ddgclib._plotting import *


from interruptingcow import timeout




In [10]:
def analytical_cap(r, theta_p):
    R = r / np.cos(theta_p)
    K_f = (1 / R) ** 2
    H_f = 2 / R  # 2 / R
    h = np.abs(R * np.sin(theta_p) - R)  # Height of the spherical cap
    # Area of spherical cap:
    dA = 2 * np.pi * R * h
    dC = 2 * np.pi * r
    a = R * np.cos(theta_p)
    # a = R * np.sin(theta_p)
    k_g_f = np.sqrt(R ** 2 - a ** 2) / (R * a)
    return H_f, K_f, dA, k_g_f, dC

NameError: name 'theta_p' is not defined

In [11]:
'''
Comes from _curvature.py
'''

'''
Questions to Stefan:

General: What is theta_i?


Line 49: What is happening here?
        N_f0 = np.array(
            [0.0, 0.0, R * np.sin(theta_p)]) - v.x_a  # First approximation

Line 59: Calculating of the curvature at the vertex 'v'
        c_outd = b_curvatures_hn_ij_c_ij(F, nn, n_i=N_f0) # curvature_dict - calculating the curvature at the vertex v
'''

'''
Comments from Flo:
K_f = mean curvature
H_f = Gaussian curvature

if 0: = boolean for false

v.x[2] = number of the x-coordiante
v.x_a = hole coordiante of x-coordiante

using b_curvature
'''

def HC_curvatures(HC, bV, r, theta_p, printout=False):
    R = r / np.cos(theta_p)
    K_f = (1 / R) ** 2 #
    H_f = 2 / R  # 2 / R
    HNdA_ij = []
    HN_i = []
    C_ij = []
    K_H_i = []
    HNdA_i_Cij = []
    Theta_i = []

    N_i = []  # Temp cap rise normal

    HNdA_i_cache = {}
    HN_i_cache = {}
    C_ij_cache = {}
    K_H_i_cache = {}
    HNdA_i_Cij_cache = {}
    Theta_i_cache = {}

    for v in HC.V:
        #TODO: REMOVE UNDER NORMAL CONDITIONS:
        if 0: # boolean operatore for =false
            if v in bV:
                continue
        N_f0 = np.array(
            [0.0, 0.0, R * np.sin(theta_p)]) - v.x_a  # First approximation
        # for bridges this should be something like:
        # N_f0 = v.x_a - np.array([v.x_a[0], v.x_a[1], 0.0])
        N_f0 = normalized(N_f0)[0]
        N_i.append(N_f0)
        F, nn = vectorise_vnn(v)
        # Compute discrete curvatures
        c_outd = b_curvatures_hn_ij_c_ij(F, nn, n_i=N_f0) # curvature_dict - calculating the curvature at the vertex v
        # Append lists
        HNdA_ij.append(c_outd['HNdA_i'])
        #HNdA_ij_dot.append(np.sum(np.dot(c_outd['HNdA_ij'], c_outd['n_i'])))
        HN_i.append(c_outd['HN_i'])
        C_ij.append(c_outd['C_ij'])
        K_H_i.append(c_outd['K_H_i'])
        HNdA_i_Cij.append(c_outd['HNdA_ij_Cij'])
        Theta_i.append(c_outd['theta_i'])

        # Append chace
        HNdA_i_cache[v.x] = c_outd['HNdA_i']
        HN_i_cache[v.x] = c_outd['HN_i']
        C_ij_cache[v.x] = c_outd['C_ij']
        K_H_i_cache[v.x] = c_outd['K_H_i']
        HNdA_i_Cij_cache[v.x] = c_outd['HNdA_ij_Cij']
        Theta_i_cache[v.x] = c_outd['theta_i']

    if printout:
        print('.')
        print(f'HNdA_ij = {HNdA_ij}')
        print(f'HN_i = {HN_i}')
        print(f'C_ij = {C_ij}')
        print(f'K_H_i = {K_H_i}')
        print(f'HNdA_i_Cij = {HNdA_i_Cij}')
        print(f'Theta_i= {Theta_i}')
        print(f'np.array(Theta_i) in deg = {np.array(Theta_i) *180/np.pi}')
        print(f'np.array(Theta_i)/np.pi= {np.array(Theta_i) / np.pi}')
        # s = r * theta
        # circ = 2 pi r
        # circ / s = 2 pi / theta
        rati = 2 * np.pi /np.array(Theta_i)
        rati = 2 * np.pi / (2 * np.pi - np.array(Theta_i))
        rati =  2 * np.pi / (2 * np.pi - np.array(Theta_i))
        rati =  (np.pi - np.array(Theta_i)/ 2 * np.pi )
        #rati = np.array(Theta_i) / (2 * np.pi)
        #print(f' rati = 2 * np.pi /np.array(Theta_i)= { rati}')
        print(f' rati = { rati}')
       # rati =  (2 * np.pi  - np.array(Theta_i))/np.pi
        #print(f' rati = (2 * np.pi  - Theta_i)/np.pi = { rati}')
        print(f'HNdA_i[1] * rati[1]  = {HNdA_ij[1] * rati[1] }')
        print(f'C_ij   = {C_ij }')
        #print(f'np.sum(C_ij)   = {np.sum(C_ij) }')
       # print(f'HNdA_i / np.array(C_ijk)  = {HNdA_i  / np.array(C_ijk)}')

        print('.')
        print(f'HNdA_i_Cij = {HNdA_i_Cij}')

        #print(f'K_H = {K_H}')
        print('-')
        print('Errors:')
        print('-')
       # print(f'hnda_i_sum = {hnda_i_sum}')
       # print(f'K_H_2 = {K_H_2}')
       # print(f'C_ijk = {C_ijk}')
      #  print(f'np.sum(C_ijk) = {np.sum(C_ijk)}')
      #  print(f'A_ijk = {A_ijk}')
     #   print(f'np.sum(A_ijk) = {np.sum(A_ijk)}')
      #  print(f'K_H_2 dA = {np.sum(K_H_2) * np.sum(C_ijk)}')
      #  print(f'K_H_2 dA = {np.sum(K_H_2) * np.sum(A_ijk)}')
      #  print(f'np.sum(K_H_2) = {np.sum(K_H_2)}')
      #  print(f'HNdA_ij_dot_hnda_i = {np.array(HNdA_ij_dot_hnda_i)}')
     #   print(
    #        f'np.sum(HNdA_ij_dot_hnda_i) = {np.sum(np.array(HNdA_ij_dot_hnda_i))}')

        print(f'K_H_i - K_f = {np.array(K_H_i) - K_f}')
        #print(f'HNdA_ij_dot_hnda_i  - H_f = {HNdA_ij_dot_hnda_i - H_f}')
        #print(f'HHN_i  - H_f = {HNdA_ij_dot_hnda_i - H_f}')
        print(f'HN_i  - H_f = {HN_i - H_f}')
        print(f'HNdA_i_Cij  - H_f = {HNdA_i_Cij - H_f}')

        #print(f'np.sum(C_ij) = {np.sum(C_ij)}')

    return (HN_i, C_ij, K_H_i, HNdA_i_Cij, Theta_i,
            HNdA_i_cache, HN_i_cache, C_ij_cache, K_H_i_cache, HNdA_i_Cij_cache,
            Theta_i_cache)

# Caprise Flow

After this point, the code comes from caprise_flow_Flo.py

In [12]:
'''
Questions to Stefan:

General: What is the Euler characteristic?
'''

'''
Comments from Flo:

Xi = Euler characteristic

'''
def mean_flow(HC, bV, params, tau, print_out=False):
    (gamma, rho, g, r, theta_p, K_f, h) = params
    print('.')
    # Compute interior curvatures
    #(HNda_v_cache, K_H_cache, C_ijk_v_cache, HN_i, HNdA_ij_dot_hnda_i,
    # K_H_2, HNdA_i_Cij) = int_curvatures(HC, bV, r, theta_p, printout=False)
    (HN_i, C_ij, K_H_i, HNdA_i_Cij, Theta_i,
     HNdA_i_cache, HN_i_cache, C_ij_cache, K_H_i_cache, HNdA_i_Cij_cache,
     Theta_i_cache) = HC_curvatures(HC, bV, r, theta_p, printout=0)

    # if bV is None:
    #bV = HC.boundary()  #TODO: Check it again it is not working properly
    # Move boundary vertices:
    bV_new = set()
    for v in HC.V:
        # Compute boundary movements
        if v in bV:
            rati = (np.pi - np.array(Theta_i) / 2 * np.pi)
            #TODO: THis is not the actual sector ration (wrong angle)
            # rati = np.array(Theta_i) / (2 * np.pi)
            # print(f' rati = 2 * np.pi /np.array(Theta_i)= { rati}')

            if 0:
                #TODO: len(bV) is sector fraction
                H_K = HNda_v_cache[v.x] * np.array([0, 0, -1]) * len(bV)
                print(f'K_H in bV = {H_K }')

                K_H = ((np.sum(H_K) / 2.0) / C_ijk_v_cache[v.x] ) ** 2
                K_H = ((np.sum(H_K) / 2.0)  ) ** 2
                print(f'K_H in bV = {K_H}')
                print(f'K_H - K_f in bV = {K_H - K_f}')

            K_H_dA = K_H_i_cache[v.x] * np.sum(C_ij_cache[v.x])
            #TODO: Adjust for other geometric approximations:
            # for bridges should always replace len(bV) with (len(bV)/2)
            l_a = 2 * np.pi * r /  len(bV)  # arc length
            Xi = 1
            # Gauss-Bonnet: int_M K dA + int_dM kg ds = 2 pi Xi
            # NOTE: Area should be height of spherical cap
            # h = R - r * 4np.tan(theta_p)
            # Approximate radius of the great shpere K = (1/R)**2:
            #R_approx = 1 / np.sqrt(K_f)
            R_approx = 1 / np.sqrt(K_H_i_cache[v.x])
            theta_p_approx = np.arccos(np.min([r / R_approx, 1]))
            h = R_approx - r * np.tan(theta_p_approx)
            A_approx = 2 * np.pi * R_approx * h  # Area of spherical cap

            #print(f'A_approx = {A_approx}')
            # A_approx  # Approximate area of the spherical cap
            #kg_ds = 2 * np.pi * Xi - K_f * (A_approx)
            #kg_ds = 2 * np.pi * Xi - K_H_dA * (A_approx)
            kg_ds = 2 * np.pi * Xi - K_H_i_cache[v.x] * (A_approx)

            # TODO: This is NOT the correct arc length (wrong angle)
            ds = 2 * np.pi * r  # Arc length of whole spherical cap
            #print(f'ds = {ds}')
            k_g = kg_ds / ds  # / 2.0
            #print(f'k_g = {k_g}')
            print(f' R_approx * k_g = {R_approx * k_g}')
            phi_est = np.arctan(R_approx * k_g)


            # Compute boundary forces
            # N m-1
            print(f' phi_est = { phi_est}')
            print(f' theta_p = {theta_p}')
            gamma_bt = gamma * (np.cos(phi_est)
                                - np.cos(theta_p)) #* np.array([0, 0, 1.0])

            print(f' phi_est = {phi_est * 180/np.pi}')
            F_bt = gamma_bt * l_a  # N, magnitude force acting on the three-phase contact line
            # To get e_ij and e_ik
            # for vn in v.nn:
            #    if vn in bV:
            #        #this is e_ij or e_ik
            # compute the cross product (wedge) and normalise, this is the direction for F_bt
            print(f' F_bt = {F_bt}')
            #new_vx = v.x + tau * F_bt
            new_vx = v.x + 1e-1 * F_bt
            new_vx = tuple(new_vx)

            if print_out:
                print('.')
                print(f'K_H_i_cache[v.x = {v.x}] = {K_H_i_cache[v.x]}')
                print(f'HNdA_i_cache[v.x = {v.x}] = {HNdA_i_cache[v.x]}')
                print(f' rati = {rati}')
                # rati =  (2 * np.pi  - np.array(Theta_i))/np.pi
                # print(f' rati = (2 * np.pi  - Theta_i)/np.pi = { rati}')
                print(
                    f'HNdA_i_cache[1] * rati[1]  = {HNdA_i_cache[v.x] * rati[1]}')

                print(f'K_H_i = {K_H_i}')
                print(f'K_f = {K_f}')
                print(f'K_H_i_cache[v.x] = {K_H_i_cache[v.x]}')

            HC.V.move(v, new_vx)
            bV_new.add(HC.V[new_vx])

            if print_out:
                print(f'K_H_dA= {K_H_dA}')
                print(f'l_a= {l_a}')
                print(f'R_approx = {R_approx}')
                print(f'theta_p_approx = {theta_p_approx * 180 / np.pi}')
                print(f'Theta_i = {Theta_i}')
                print(f'phi_est  = {phi_est * 180 / np.pi}')
                #print(f'dK[i] = {dK[i]}')
        else:

            #H = np.dot(HNdA_i_cache[v.x], np.array([0, 0, 1]))
            H = HN_i_cache[v.x] #TODO: Why is this sometimes negative? Should never be
            #H = np.abs(H)
            print(f' H = {H}')
            print(f' np.dot(HN_i_cache[v.x], np.array([0, 0, 1])) = {np.dot(HN_i_cache[v.x], np.array([0, 0, 1]))}')
            print(f' HN_i_cache[v.x] = {HN_i_cache[v.x]}')
            print(f' H = {H}')
            #

            height = np.max([v.x_a[2], 0.0])
            df = gamma * H  # Hydrostatic pressure
            print(f' gamma * H = { gamma * H}')
            print(f' HNdA_i_cache[v.x] = {HNdA_i_cache[v.x]}')
            print(f' HN_i = {HNdA_i_cache[v.x]}')
            print(f' rho * g * height = {rho * g * height}')
            print(f' height = {height}')
            df = gamma * H - (rho * g * height)
            #df = 2* gamma * H - 1e-3*(rho * g * height)
            #f_k = f + tau * df

            f_k = v.x_a + np.array([0, 0, tau * df])
            f_k[2] = np.max([f_k[2], 0.0])
            new_vx = tuple(f_k)

            #VA.append(v.x_a)
            # Move interior complex
            if print_out:
                print('.')
                print(f'HNdA_i_cache[{v.x}] = {HNdA_i_cache[v.x]}')
                print(f'HN_i_cache[{v.x}] = {HN_i_cache[v.x]}')
                print(f'H = {H}')
                print(f'v.x_a  = {v.x_a}')
                print(f'df = {df}')
                print(f'height = {height}')

                print(f'np.max([f_k[2], 0.0]) = {np.max([f_k[2], 0.0])}')
                print(f'f_k = {f_k}')

            HC.V.move(v, new_vx)


    if print_out:
        print(f'bV_new = {bV_new}')

    return HC, bV_new

In [13]:
def cap_rise_init_N(r, theta_p, gamma, N=4, refinement=0, cdist=1e-10,
                    equilibrium=True):
    Theta = np.linspace(0.0, 2 * np.pi, N)  # range of theta
    R = r / np.cos(theta_p)  # = R at theta = 0
    # Exact values:
    K_f = (1 / R) ** 2  # Ga
    H_f = 1 / R + 1 / R  # 2 / R
    dp_exact = gamma * (2 / R)  # Pa      # Young-Laplace equation  dp = - gamma * H_f = - gamma * (1/R1 + 1/R2)
    F = []
    nn = []
    F.append(np.array([0.0, 0.0, R * np.sin(theta_p) - R]))
    nn.append([])
    ind = 0
    for theta in Theta:
        ind += 1
        # Define coordinates:
        # x, y, z = sphere(R, theta, phi)
        F.append(np.array([r * np.sin(theta), r * np.cos(theta), 0.0]))
        # Define connections:
        nn.append([])
        if ind > 0:
            nn[0].append(ind)
            nn[ind].append(0)
            nn[ind].append(ind - 1)
            nn[ind].append((ind + 1) % N)

    # clean F
    for f in F:
        for i, fx in enumerate(f):
            if abs(fx) < 1e-15:
                f[i] = 0.0

    F = np.array(F)
    nn[1][1] = ind

    # Construct complex from the initial geometry:
    HC = construct_HC(F, nn)
    v0 = HC.V[tuple(F[0])]
    # Compute boundary vertices
    V = set()
    for v in HC.V:
        V.add(v)
    bV = V - set([v0])
    for i in range(refinement):
        V = set()
        for v in HC.V:
            V.add(v)
        HC.refine_all_star(exclude=bV)
        # New boundary vertices:
        for v in HC.V:
            if v.x[2] == 0.0:
                bV.add(v)

    # Move to spherical cap
    for v in HC.V:
        z = v.x_a[2]
        z_sphere = z - R * np.sin(theta_p)  # move to origin
        # z_sphere = R * np.cos(phi)  # For a sphere centered at origin
        phi_v = np.arccos(z_sphere/R)
        plane_dist = R * np.sin(phi_v)
        # Push vertices on the z-slice the required distance
        z_axis = np.array([0.0, 0.0, z])  # axial centre
        vec = v.x_a - z_axis
        s = np.abs(np.linalg.norm(vec) - plane_dist)
        nvec = normalized(vec)[0]
        nvec = v.x_a + s * nvec
        HC.V.move(v, tuple(nvec))
        vec = nvec - z_axis
        np.linalg.norm(vec)

    # Rebuild set after moved vertices (appears to be needed)
    bV = set()
    for v in HC.V:
        if v.x[2] == 0.0:
            bV.add(v)

    if not equilibrium:
        # Move to zero, for mean flow simulations
        VA = []
        for v in HC.V:
            if v in bV:
                continue
            else:
                VA.append(v.x_a)

        VA = np.array(VA)
        for i, v_a in enumerate(VA):
            v = HC.V[tuple(v_a)]
            v_new = tuple(v.x_a - np.array([0.0, 0.0, v.x_a[2]]))
            HC.V.move(v, v_new)

    if 0:
        R = r / np.cos(theta_p)  # = R at theta = 0
        K_f = (1 / R) ** 2
        H_f = 1 / R + 1 / R  # 2 / R
        rho = 1000  # kg/m3, density
        g = 9.81  # m/s2
        h_jurin = 2 * gamma * np.cos(theta_p) / (rho * g * r)

        for v in HC.V:
            #v = HC.V[tuple(v_a)]
            v_new = tuple(v.x_a + np.array([0.0, 0.0, h_jurin]))
            HC.V.move(v, v_new)

    # TODO: Reconstruct F, nn
   # F = []
   # nn = []
    return F, nn, HC, bV, K_f, H_f

In [14]:
def incr(HC, bV, params, tau=1e-5, plot=False, verbosity=1):
    HC.dim = 3  # Rest in case visualization has changed

    if verbosity == 2:
        print_out = True
    else:
        print_out = False
    # Update the progress
    HC, bV = mean_flow(HC, bV, params, tau=tau, print_out=print_out)

    # Compute progress of Capillary rise:
    if verbosity == 1:
        print('.')
        current_Jurin_err(HC)

    # Plot in Polyscope:
    if plot:
        pass
        #ps_inc(surface, HC)
        #HC.plot_complex()
        #plt.close

    return HC, bV

In [15]:
def current_Jurin_err(HC):
    #h_final = 0.0
    h_final = np.inf
    v_min = np.inf
    for v in HC.V:
        #h_final = np.max([h_final, v.x_a[2]])
        v_min = np.min([v_min, v.x_a[2]])
        #h_final = np.max([h_final, v_min])
        h_final = np.min([h_final, v_min])
    # print(f'h_final = {h} m')
    print(f'h_final = {h_final} m')
    print(f'h (Jurin) = {h_jurin} m')
    print(f'h_final - h (Jurin) = {h_final - h_jurin} m')
    try:
        print(f'Error: h_final - h (Jurin) = {100 * abs((h_final - h_jurin)/ h_jurin)} %')
    except ZeroDivisionError:
        pass
    return

In [18]:
r = 1

if 1:
    # Define HC
    # Compute analytical ratio
    H_f, K_f, dA, k_g_f, dC = analytical_cap(r, theta_p)
    print(f'H_f, K_f, dA, k_g_f, dC = {H_f, K_f, dA, k_g_f, dC}')
    k_K = k_g_f / K_f  #TODO: We no longer need this?
    h_jurin = 2 * gamma * np.cos(theta_p) / (rho * g * r)
    print(f'h_jurin = {h_jurin}')

    # Prepare film and move it to 0.0
    F, nn, HC, bV, K_f, H_f = cap_rise_init_N(r, theta_p, gamma, N=N,
                                              refinement=refinement,
                                              equilibrium=equilibrium
                                              )

    #h = 0.0
    params = (gamma, rho, g, r, theta_p, K_f, h)
    # HC.V.print_out()
    HC.V.merge_all(cdist=cdist)

    # For picturing solution:
    if 0:
        for v in HC.V:
            new_vx = v.x_a
            new_vx[2] = v.x[2] + h_jurin
            HC.V.move(v, tuple(new_vx))

    # Escaping saddle points midway, test for equilibria:
    if 0:
        for v in HC.V:
            abv = False
            if v in bV:
                abv = True
            new_vx = v.x_a
            #new_vx[2] = v.x[2] + 0.9*h_jurin
            new_vx[2] = v.x[2] + h #0.9*h_jurin
            HC.V.move(v, tuple(new_vx))
            if abv:
                bV.add(set([HC.V[tuple(new_vx)]]))
    # HC, bV = kmean_flow(HC, bV, params, tau=tau, print_out=0)

if 1:
    #NOTE: Does NOT work after running through incr at all, only works at
    #      equilibrium
    plot_polyscope(HC)

NameError: name 'theta_p' is not defined

In [19]:
def ps_inc(surface, HC):
    #F, nn, HC, bV, K_f, H_f = cap_rise_init_N(r, theta_p, gamma, N=N,
    #                                          refinement=refinement,
    #                                          equilibrium=True
    #                                          )

    HC.dim = 2  # The dimension has changed to 2 (boundary surface)
    HC.vertex_face_mesh()
    points = np.array(HC.vertices_fm)
    triangles = np.array(HC.simplices_fm_i)

    ### Register a point cloud
    # `my_points` is a Nx3 numpy array
    my_points = points
    ps_cloud = ps.register_point_cloud("my points", my_points)
    # ps_cloud.set_color((0.0, 0.0, 0.0))
    verts = my_points
    newPositions = verts
    surface.update_vertex_positions(newPositions)
    try:
        with timeout(0.1, exception=RuntimeError):
            # perform a potentially very slow operation
            ps.show()
    except RuntimeError:
        pass

In [20]:

# Matplotlib and plotly
if 0:
    print('Finished, plotting...')
    if 0:
        HC.plot_complex()
        plt.show()


    # First possibility: plotly
    if 1:
        import plotly.graph_objects as go

        fig = go.Figure(data=[
            go.Mesh3d(
                x=points[:, 0],
                y=points[:, 1],
                z=points[:, 2],
                i=triangles[:, 0],
                j=triangles[:, 1],
                k=triangles[:, 2],
            )
        ])
        fig.show()
    # Second possibility: matplotlib
    if 1:

        from mpl_toolkits.mplot3d import Axes3D
        import matplotlib.pyplot as plt

        fig = plt.figure()
        ax = fig.gca(projection='3d')
        ax.plot_trisurf(points[:, 0], points[:, 1], points[:, 2],
                        triangles=triangles)
        plt.show()


In [21]:
def plot_polyscope(HC):
    # Initialize polyscope
    ps.init()
    ps.set_up_dir("z_up")

    do = coldict['db']
    lo = coldict['lb']
    HC.dim = 2  # The dimension has changed to 2 (boundary surface)
    HC.vertex_face_mesh()
    points = np.array(HC.vertices_fm)
    triangles = np.array(HC.simplices_fm_i)
    print(f'points = {points}')
    print(f'triangles = {triangles}')
    ### Register a point cloud
    # `my_points` is a Nx3 numpy array
    my_points = points
    ps_cloud = ps.register_point_cloud("my points", my_points)
    ps_cloud.set_color(tuple(do))
    #ps_cloud.set_color((0.0, 0.0, 0.0))
    verts = my_points
    faces = triangles
    ### Register a mesh
    # `verts` is a Nx3 numpy array of vertex positions
    # `faces` is a Fx3 array of indices, or a nested list
    surface = ps.register_surface_mesh("my mesh", verts, faces,
                             color=do,
                             edge_width=1.0,
                             edge_color=(0.0, 0.0, 0.0),
                             smooth_shade=False)

    # Add a scalar function and a vector function defined on the mesh
    # vertex_scalar is a length V numpy array of values
    # face_vectors is an Fx3 array of vectors per face
    if 0:
        ps.get_surface_mesh("my mesh").add_scalar_quantity("my_scalar",
                vertex_scalar, defined_on='vertices', cmap='blues')
        ps.get_surface_mesh("my mesh").add_vector_quantity("my_vector",
                face_vectors, defined_on='faces', color=(0.2, 0.5, 0.5))

    # View the point cloud and mesh we just registered in the 3D UI
    #ps.show()
    ps.show()


if 0:

    # Initialize polyscope
    ps.init()
    ps.set_up_dir("z_up")

    do = coldict['db']
    lo = coldict['lb']
    HC.dim = 2  # The dimension has changed to 2 (boundary surface)
    HC.vertex_face_mesh()
    points = np.array(HC.vertices_fm)
    triangles = np.array(HC.simplices_fm_i)
    ### Register a point cloud
    # `my_points` is a Nx3 numpy array
    my_points = points
    ps_cloud = ps.register_point_cloud("my points", my_points)
    ps_cloud.set_color(tuple(do))
    #ps_cloud.set_color((0.0, 0.0, 0.0))
    verts = my_points
    faces = triangles
    ### Register a mesh
    # `verts` is a Nx3 numpy array of vertex positions
    # `faces` is a Fx3 array of indices, or a nested list
    surface = ps.register_surface_mesh("my mesh", verts, faces,
                             color=do,
                             edge_width=1.0,
                             edge_color=(0.0, 0.0, 0.0),
                             smooth_shade=False)

    # Add a scalar function and a vector function defined on the mesh
    # vertex_scalar is a length V numpy array of values
    # face_vectors is an Fx3 array of vectors per face
    if 1:
        ps.get_surface_mesh("my mesh").add_scalar_quantity("my_scalar",
                vertex_scalar, defined_on='vertices', cmap='blues')
        ps.get_surface_mesh("my mesh").add_vector_quantity("my_vector",
                face_vectors, defined_on='faces', color=(0.2, 0.5, 0.5))

    # View the point cloud and mesh we just registered in the 3D UI
    #ps.show()
    try:
        with timeout(0.3, exception=RuntimeError):
            # perform a potentially very slow operation
            ps.show()
    except RuntimeError:
        pass

#
if 0:
    steps = 800 # Still stable
   # steps = 800 # Still stable
    #steps = 200 # Unstable, but can escape local equilibria
    #steps = 2 # Unstable, but can escape local equilibria
    #steps = 20000 # Unstable, but can escape local equilibria
    #steps = 100000 # Unstable, but can escape local equilibria
    for i in range(steps):  #unstable
        #HC, bV = incr(HC, bV, params, tau=0.1, plot=0)
        #, bV = incr(HC, bV, params, tau=0.000001, plot=0)
        #HC, bV = incr(HC, bV, params, tau=0.0000001, plot=0)
        #HC, bV = incr(HC, bV, params, tau=0.0000001, plot=0)
        HC, bV = incr(HC, bV, params, tau=0.1, plot=0)


    if 0:
        HC.dim = 3
        HC.plot_complex()
    #plt.show()

    #ps_inc(surface, HC)

    #plot_polyscope(HC)
    plt.show()




In [42]:
# Parameters for a water droplet in air at standard laboratory conditions
gamma = 0.0728  # N/m, surface tension of water at 20 deg C
rho = 1000  # kg/m3, density
g = 9.81  # m/s2

# Parameters from EoS:
#T_0 = 273.15 + 25  # K, initial tmeperature
#P_0 = 101.325  # kPa, Ambient pressure
#gamma = IAPWS(T_0)  # N/m, surface tension of water at 20 deg C
#rho_0 = eos(P=P_0, T=T_0)  # kg/m3, density

# Capillary rise parameters
#r = 0.5e-2  # Radius of the droplet sphere
r = 2.0  # Radius of the droplet sphere
r = 2.0e-6  # Radius of the tube
r = 2.0e-6  # Radius of the tube
r = 0.5e-6  # Radius of the tube
r = 1.4e-5  # Radius of the tube
r = 1.4e-5  # Radius of the tube
r = 1e-3  # Radius of the tube (1 mm)
r = 2
#r = 2e-3  # Radius of the tube (2 mm)
#r = 20e-3  # Radius of the tube (20 mm)
#r = 1.0  # Radius of the droplet sphere
#r = 10.0  # Radius of the droplet sphere
#r = 1.0  # Radius of the droplet sphere
#r = 0.1  # Radius of the droplet sphere

# h (Jurin) = 0.013946915961919293 m

h = 0.0130  # Initial film height (TODO: Set higher)


h = 1  # Initial film height (TODO: Set higher)
#r = 0.5e-5  # Radius of the droplet sphere
theta_p = 45 * np.pi/180.0  # Three phase contact angle
theta_p = 20 * np.pi/180.0  # Three phase contact angle
theta_p = 45 * np.pi/180.0  # Three phase contact angle
theta_p = 45 * np.pi/180.0  # Three phase contact angle

#phi = 0.0
N = 8
N = 6
N = 6
#N = 7
#N = 5
#N = 12
#refinement = 2#2
refinement = 0#2
refinement = 2#2

#refinement = 2
equilibrium = 1#$False
#N = 20
#cdist = 1e-10
cdist = 1e-10

r = np.array(r, dtype=np.longdouble)
theta_p = np.array(theta_p, dtype=np.longdouble)


In [43]:
if 0:
    (HNda_v_cache, K_H_cache, C_ijk_v_cache, HN_i,  HNdA_ij_dot_hnda_i,
     K_H_2, HNdA_i_Cij) = int_curvatures(HC, bV, r, theta_p, printout=0)
    # Laplacian error
    print(f'H_f - HN_i = {H_f - np.array(HN_i)}')
    #print(f'HN_i = {}')



In [44]:
# Define HC
'''
# Compute analytical ratio
H_f, K_f, dA, k_g_f, dC = analytical_cap(r, theta_p)
print(f'H_f, K_f, dA, k_g_f, dC = {H_f, K_f, dA, k_g_f, dC}')
k_K = k_g_f / K_f  #TODO: We no longer need this?
h_jurin = 2 * gamma * np.cos(theta_p) / (rho * g * r)
'''

# Prepare film and move it to 0.0
F, nn, HC, bV, K_f, H_f = cap_rise_init_N(r, theta_p, gamma, N=N,
                                          refinement=refinement,
                                          equilibrium=equilibrium
                                          )

#h = 0.0
params = (gamma, rho, g, r, theta_p, K_f, h)
# HC.V.print_out()
HC.V.merge_all(cdist=cdist)


plot_polyscope(HC)

np.zeros([self.V.size(), self.dim]) = [[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
self.V[v].index = 0
self.V[v].index = 1
self.V[v].index = 2
self.V[v].index = 3
self.V[v].index = 4
self.V[v].index = 5
self.V[v].index = 6
self.V[v].index = 7
self.V[v].index = 8
self.V[v].index = 9
self.V[v].index = 10
self.V[v].index = 11
self.V[v].index = 12
self.V[v].index = 13
self.V[v].index = 14
self.V[v].index = 15
self.V[v].index = 16
self.V[v].index = 17
self.V[v].index = 18
self.V[v].index = 19
self.V[v].index = 20
self.V[v].index = 21
self.V[v].index = 22
self.V[v].index = 23
self.V[v].index = 24
self.V[v].index = 25
self.V[v].index = 26
self.V[v].index = 27
self.V[v].index = 28
self.V[v].index = 29
self.V[v].index = 30
points = [[ 0.00000000e+00  0.00000000e+0